In [20]:
import scipy.stats as st
import pandas as pd 
import seaborn as sns
from pylab import mpl 
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()
import warnings

warnings.filterwarnings('ignore')
# plt.rcParams['font.sans-serif'] = ['STSong']
# mpl.rcParams['font.sans-serif'] = ['STSong'] # 指定默认字体 
mpl.rcParams['axes.unicode_minus'] = False


In [21]:
df1 = pd.read_csv('./data/file1.csv')
df4 = pd.read_csv('./data/file4.csv')

df5 = pd.read_csv('./data/file5.csv')


# 将最新的定价作为price 和提取降价次数、幅度特征

In [22]:
# 将最后一次调整价格的时间，作为pushDate，最新的价格作为pushPrice
def clear_time(txt_json):
    if txt_json == '{}':
        return np.nan
    else:
        update_time = txt_json[2:-2].split(',')[-1].split(':')[0].strip()[2:-2]
        update_price = txt_json[2:-
                                2].split(',')[-1].split(':')[1].strip()[2:-2]
        barging_times = len(txt_json[2:-2].split(','))
        new_str = update_time+','+update_price+','+str(barging_times)
        return new_str
# 对file4进行处理

df4['updatePriceTimeJson'] = df4['updatePriceTimeJson'].progress_apply(
    clear_time)
sep = df4['updatePriceTimeJson'].astype(str).str.split(',', expand=True)
df4['update_time'] = sep[0].astype(str)
df4['update_price'] = sep[1].astype(float)
df4['update_price'] = df4['update_price'].fillna(0)

# 提取降价次数特征
df4['barging_times'] = sep[2].astype(float)
# 提取降价幅度特征
df4['barging_price'] = np.nan
for i in range(df4.shape[0]):
    update_time = df4['update_time'][i]
    update_price = df4['update_price'][i]
    if update_price == 0:
        df4['barging_price'][i] = 0
    else:
        df4['barging_price'][i] = df4['pushPrice'][i]-update_price
        df4['pushDate'][i] = update_time
        df4['pushPrice'][i] = update_price


100%|██████████| 10000/10000 [00:00<00:00, 208276.01it/s]


In [23]:
df4

,carid,pushDate,pushPrice,updatePriceTimeJson,pullDate,withdrawDate,update_time,update_price,barging_times,barging_price
0,68603,2021-03-11,3.9800,NaN,2021-03-11,2021-03-11,nan,0.0,NaN,0.00
1,12312,2021-05-14,4.5000,NaN,2021-06-14,2021-06-14,nan,0.0,NaN,0.00
2,57655,2021-04-05,23.0000,"2021-04-05,23,1",2021-04-08,2021-04-08,2021-04-05,23.0,1.0,0.90
3,45688,2020-09-01,20.5798,NaN,2020-09-04,2020-09-04,nan,0.0,NaN,0.00
4,52081,2021-05-20,11.9000,"2021-05-20,11.9,1",2021-06-21,2021-06-21,2021-05-20,11.9,1.0,0.38
...,...,...,...,...,...,...,...,...,...,...
9995,10177,2021-07-31,22.8800,NaN,2021-09-11,NaN,nan,0.0,NaN,0.00
9996,54312,2021-03-22,12.3000,NaN,2021-04-16,NaN,nan,0.0,NaN,0.00
9997,45597,2021-01-09,15.3800,NaN,2021-01-27,NaN,nan,0.0,NaN,0.00
9998,38629,2021-06-12,36.6000,"2021-06-12,36.6,1",2021-06-22,NaN,2021-06-12,36.6,1.0,0.28


In [24]:
df_11 = df1[['carid','modelyear','newprice']]
df4 = df_11.merge(df4,on='carid')

# 1 数据预处理

In [25]:

# 去除没卖出的样本
df_trans = df4[df4.withdrawDate.notna() & df4.modelyear.notna() & df4.newprice.notna() ]

## 1.1 计算交易周期

In [26]:
df_trans['pushDate'] = pd.to_datetime(df_trans['pushDate'])
df_trans['withdrawDate'] = pd.to_datetime(df_trans['withdrawDate'])
trans_circle = pd.DataFrame(df_trans['withdrawDate'] - df_trans['pushDate'])
df_trans['transcycle'] = trans_circle[0]
# 转为整型
sep = df_trans['transcycle'].astype(str).str.split(' ', expand=True)
df_trans['transcycle'] = sep[0].astype(int)

In [28]:
data_df = df_trans[['carid','withdrawDate','pushPrice','modelyear','transcycle','newprice']]
data_df.to_csv('./submit/问题二分析需要的文件.csv',index=0)

In [29]:
data_df

,carid,withdrawDate,pushPrice,modelyear,transcycle,newprice
0,2,2021-07-23,7.38,2017.0,28,11.98
1,6,2021-06-30,4.38,2016.0,1,9.49
2,7,2021-07-19,5.90,2012.0,19,18.08
3,10,2021-06-08,1.49,2006.0,0,11.25
4,12,2021-06-16,0.30,2010.0,0,4.18
...,...,...,...,...,...,...
7995,74096,2021-07-10,10.98,2019.0,2,14.08
7996,74116,2021-04-19,3.20,2009.0,2,25.18
7997,74136,2021-01-27,13.50,2018.0,7,21.98
7998,74140,2021-01-13,14.30,2018.0,18,21.98
